In [13]:
import yfinance as yf


data = yf.Ticker('SPY')

In [ ]:
data.info
data.calendar
data.analyst_price_targets
data.quarterly_income_stmt
data.history(period='1d', interval='1m')
data.options

options_history = {}
for option_date in data.options:
    option_chain = data.option_chain(option_date).calls
    for option in option_chain.itertuples(index=False):
        if option.contractSymbol not in options_history:
            options_history[option.contractSymbol] = []
        options_history[option.contractSymbol].append(option.lastPrice)
print(options_history)

{'SPY251229C00500000': [190.45], 'SPY251229C00580000': [96.31], 'SPY251229C00595000': [94.54], 'SPY251229C00600000': [78.54], 'SPY251229C00605000': [84.79], 'SPY251229C00610000': [62.69], 'SPY251229C00615000': [74.83], 'SPY251229C00620000': [70.56], 'SPY251229C00625000': [59.3], 'SPY251229C00630000': [60.64], 'SPY251229C00635000': [55.64], 'SPY251229C00640000': [50.14], 'SPY251229C00645000': [46.43], 'SPY251229C00650000': [41.52], 'SPY251229C00651000': [39.2], 'SPY251229C00653000': [31.42], 'SPY251229C00654000': [22.29], 'SPY251229C00655000': [35.24], 'SPY251229C00656000': [20.51], 'SPY251229C00657000': [34.61], 'SPY251229C00658000': [23.69], 'SPY251229C00659000': [28.0], 'SPY251229C00660000': [30.2], 'SPY251229C00661000': [25.83], 'SPY251229C00662000': [28.71], 'SPY251229C00663000': [27.07], 'SPY251229C00664000': [15.58], 'SPY251229C00665000': [25.46], 'SPY251229C00666000': [24.89], 'SPY251229C00667000': [14.98], 'SPY251229C00668000': [12.98], 'SPY251229C00669000': [20.19], 'SPY251229